In [1]:
# ライブラリimport

from socket import socket, AF_INET, SOCK_DGRAM
from struct import *
from datetime import *

def print_bin(label, bin):
	print(label, end=" : ")
	for i in bin:
		print(format(i, "02x"), end=" ")
	print("")

# UDPソケット

SOCKET_BUFFSIZE = 2048
ETHIO_SERVER = ('192.168.1.204', 16241)
sock = socket(AF_INET, SOCK_DGRAM)

# コマンドパケット作成（データバイト長は1472以下であること）

ETHIO_PACKET_FOURCC = b'AVMM'
ETHIO_PACKET_ENDCMD = b'\x7f\x00\xff\xff'

def data_padding(data):
	return data + b'\x00' * ((4 - len(data)) & 3)

def cmd_memwr(addr, wrbyte):
	return pack('!BxHL', 0x44, len(wrbyte), addr) + data_padding(wrbyte)

def cmd_memrd(addr, readnum):
	return pack('!BxHL', 0x54, readnum, addr)

def cmd_fifowr(ch, wrbyte):
	return pack('!BxH', 0x20 | (ch & 15), len(wrbyte)) + data_padding(wrbyte)

def cmd_fiford(ch, readnum):
	return pack('!BxH', 0x30 | (ch & 15), readnum)


In [2]:
# NeoPixel制御用関数

def pixel(r, g, b):
	r = 0 if r<0 else 254 if r>254 else int(r)
	g = 0 if g<0 else 254 if g>254 else int(g)
	b = 0 if b<0 else 254 if b>254 else int(b)
	return pack('BBB', g,r,b)

def hsv_to_rgb(h, s, v): # h:0.0~1.0, s:0.0~1.0, v:0.0~1.0
	if s == 0.0: v*=255; return (v, v, v)
	i = int(h*6.) # XXX assume int() truncates!
	f = (h*6.)-i; p,q,t = int(255*(v*(1.-s))), int(255*(v*(1.-s*f))), int(255*(v*(1.-s*(1.-f)))); v*=255; i%=6
	if i == 0: return (v, t, p)
	if i == 1: return (q, v, p)
	if i == 2: return (p, v, t)
	if i == 3: return (p, q, v)
	if i == 4: return (t, p, v)
	if i == 5: return (v, p, q)

def rainbow(phase, bright, pixelnum):
	led_data = bytes()
	for i in range(pixelnum):
		r,g,b = hsv_to_rgb(phase, 1.0, bright)
		led_data += pixel(r,g,b)
		phase += 1.0/pixelnum

	return led_data


In [ ]:
# レインボーアニメーション

import time

phase = [0.0, 0.0, 0.0, 0.5]
speed = [0.01, 0.017, 0.05, 0.05]
pixelnum = [60, 60, 60, 60]		# チャネルあたり340個以下、全チャネル合計では470個以下にすること

while True:
	send_bin = ETHIO_PACKET_FOURCC
	for i in range(4):
		send_bin += cmd_fifowr(i, rainbow(phase[i], 0.1, pixelnum[i]))
		phase[i] += speed[i]
	send_bin += ETHIO_PACKET_ENDCMD

	sock.sendto(send_bin, ETHIO_SERVER)
	res_bin, addr = sock.recvfrom(SOCKET_BUFFSIZE)

	time.sleep(30/1000)
